<a href="https://colab.research.google.com/github/ashi-agrawal-06/Python-bot-for-Flipkart-data-scraping/blob/main/flipkart_scraper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm


def get_page_soup(url):
    try:
        page = requests.get(url)
        if page.status_code == 200:
            return BeautifulSoup(page.text, 'lxml')
        else:
            print('\n error',page.status_code)
            return None
    except:
        print("internet/domain error")
        return None


def extract_products(soup):
    cards = soup.find_all('div',{'class':'_4ddWXP'})
    if len(cards) > 0:
        print("\n",len(cards),"products found")
        return cards
    else:
        print("products not found")
        return None


def parse_rev_link(link1):
  r=requests.get(link1)
  nj=BeautifulSoup(r.text,'lxml')
  link2="https://www.flipkart.com"+nj.find_all('div',class_='_2c2kV-')[-1].next_sibling['href']
  return extract_reviews(link2)

def extract_reviews(link2):
  rev=[]
  for k in range(1,5):
    b=requests.get(f'{link2}&page={k}')
    t=BeautifulSoup(b.text,'lxml')
    for x in t.find_all('div',class_="col _2wzgFH K0kLPL"):
      #head=x.find('p',class_="_2-N8zT").text
      detail=" ".join(list(x.find('div',class_="t-ZTKy").stripped_strings)[:-1])
      #rev.append({'Head-review':head,'Detailed-review':detail})
      rev.append(detail)
  return rev


def all_specs(link1):
  req=requests.get(link1)
  specs_p=BeautifulSoup(req.text,'lxml')
  specs=specs_p.find('div',class_='_1UhVsV _3AsE0T')
  specification={}
  for i in specs.find_all('tr',class_='_1s_Smc row'):
    specification.update({i.find('td').text:i.find('li').text})
  return specification
    

def parse_product(product):
    title = product.find('a', {'class':'s1Q9rs'}).text.strip().replace(",",'')
    try:
        price = product.find('div',{'class':'_30jeq3'}).text
    except:
        price=None
    try:
        stars = product.find('div',{'class':'_3LWZlK'}).text
    except:
        stars = None

    try:
        link1 = "https://www.flipkart.com"+product.find('a', {'class':'s1Q9rs'})['href']
        reviews=parse_rev_link(link1)
    except:
        reviews = None
    
    try:
        link1 = "https://www.flipkart.com"+product.find('a', {'class':'s1Q9rs'})['href']
        specifications=all_specs(link1)
    except:
        specifications = {}

    try:
        imglink = product.find('img',{'class':'_3LWZlK'}).attrs.get('src')
    except:
        imglink = None

    specifications.update({'title':title,
            'price':price, 
            'stars':stars,
            'reviews':reviews,
            'imglink':imglink})
    
    return specifications

In [ ]:
dataset = []
for num in tqdm(range(1,2)):
    url = f'https://www.flipkart.com/search?q=mobile&otracker=search&otracker1=search&marketplace=FLIPKART&as-show=on&as=off&page={num}'
    soup = get_page_soup(url)    
    if soup:
        product_list = extract_products(soup)
        if product_list:
            for product in tqdm(product_list):
                data = parse_product(product)
                dataset.append(data)
        else:
            print("no products | closing scraper")
            continue
    else:
        print("no page data | closing scraper")
        continue
else:
    print("scraper closed")
df = pd.DataFrame(dataset)
df.to_csv('flipcart_mobiles.csv')
print("saved successfully")

  0%|          | 0/40 [00:00<?, ?it/s]


 40 products found



100%|██████████| 1/1 [06:52<00:00, 412.80s/it]

scraper closed
saved successfully


In [ ]:
df

,In The Box,Model Number,Model Name,Color,Browse Type,SIM Type,Hybrid Sim Slot,Touchscreen,OTG Compatible,Sound Enhancements,SAR Value,Display Size,Resolution,Resolution Type,GPU,Display Type,Display Colors,Other Display Features,Operating System,Processor Type,Processor Core,Primary Clock Speed,Secondary Clock Speed,Operating Frequency,Internal Storage,RAM,Expandable Storage,Supported Memory Card Type,Memory Card Slot Type,Primary Camera Available,Primary Camera,Primary Camera Features,Secondary Camera Available,Secondary Camera,Secondary Camera Features,Flash,HD Recording,Full HD Recording,Video Recording,Video Recording Resolution,...,Browser,Quick Charging,Call Log Memory,Optical Zoom,Image Editor,Call Wait/Hold,Hands Free,Video Call Support,Phone Book,Call Timer,Speaker Phone,Speed Dialing,Logs,NFC,Social Networking Phone,Instant Message,Business Phone,Removable Battery,MMS,Voice Input,Predictive Text Input,Series,FM Radio Recording,3G,Conference Call,Call Divert,USB Tethering,Keypad,Talk Time,Digital Zoom,DLNA Support,Mini HDMI Port,Dual Battery,Supported Languages,Phone Book Memory,Mobile Tracker,Keypad Type,Games,Music Player,Battery Type
0,"Handset, Adapter, Micro USB Cable, SIM Card To...",RMX2185,C11,Rich Green,Smartphones,Dual Sim,No,Yes,Yes,Dirac Sound Effect,"Head: 1.007 W/kg, Body: 0.65 W/kg",16.51 cm (6.5 inch),1600 x 720 Pixels,HD+,GE8320,HD+ In-cell LCD Display,16.7M,"20:9 Screen Ratio, 88.7% Screen-to-body Ratio,...",Android 10,Mediatek Helio G35,Octa Core,2.3 GHz,1.8 GHz,"GSM Bands: 850/900/1800/1900, WCDMA Bands: 1/5...",32 GB,2 GB,256 GB,microSD,Dedicated Slot,Yes,13MP + 2MP,"13MP + 2MP Rear Camera Setup, Sensor Sizes/Pix...",Yes,5MP Front Camera,"5MP Front Camera, Sensor Sizes/Pixel Data: 1/5...",Rear Single LED Flash,Yes,Yes,Yes,"720P (at 30 fps), 1080P (at 30fps)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Handset, Adapter, Micro USB Cable, SIM Card To...",RMX2185,C11,Rich Grey,Smartphones,Dual Sim,No,Yes,Yes,Dirac Sound Effect,"Head: 1.007 W/kg, Body: 0.65 W/kg",16.51 cm (6.5 inch),1600 x 720 Pixels,HD+,GE8320,HD+ In-cell LCD Display,16.7M,"20:9 Screen Ratio, 88.7% Screen-to-body Ratio,...",Android 10,Mediatek Helio G35,Octa Core,2.3 GHz,1.8 GHz,"GSM Bands: 850/900/1800/1900, WCDMA Bands: 1/5...",32 GB,2 GB,256 GB,microSD,Dedicated Slot,Yes,13MP + 2MP,"13MP + 2MP Rear Camera Setup, Sensor Sizes/Pix...",Yes,5MP Front Camera,"5MP Front Camera, Sensor Sizes/Pixel Data: 1/5...",Rear Single LED Flash,Yes,Yes,Yes,"720P (at 30 fps), 1080P (at 30fps)",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Handset, Power Adapter, Micro USB Cable, SIM E...",MZB07RIIN,C3,Arctic Blue,Smartphones,Dual Sim,No,Yes,Yes,NaN,"Head: 0.506 W/kg, Body: 0.833 W/kg",16.59 cm (6.53 inch),1600 x 720 Pixels,HD+,IMG PowerVR GE8320,HD+ In-cell LCD Display,NaN,"20:9 Aspect Ratio, Panda Glass Screen Protecti...",Android 10,Mediatek Helio G35,Octa Core,2.3 GHz,1.8 GHz,"GSM: B2/B3/B5/B8, WCDMA: B1/B5/B8, 4G FDD LTE:...",32 GB,3 GB,512 GB,microSD,Dedicated Slot,Yes,13MP + 2MP + 2MP,"13MP + 2MP + 2MP Rear Camera Setup, Primary (1...",Yes,5MP Front Camera,"5MP Front Camera, f/2.2 Aperture, 1.12μm Pixel...",Rear LED Flash,Yes,Yes,Yes,1080P (at 30 fps),...,Google Chrome,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Handset, Travel Adaptor (25W), Data Cable (USB...",SM-E625FZKDINS,Galaxy F62,Laser Grey,Smartphones,Dual Sim,No,Yes,Yes,NaN,NaN,17.02 cm (6.7 inch),2400 x 1080 Pixels,Full HD+,ARM Mali G76 MP12,Full HD+ Super AMOLED Display,16M,NaN,Android 11,Exynos 9825,Octa Core,2.73 GHz,1.95 GHz,NaN,128 GB,6 GB,1 TB,microSD,Dedicated Slot,Yes,64MP + 12MP + 5MP + 5MP,"Quad Rear Camera, Photography F

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Columns: 126 entries, In The Box to Battery Type
dtypes: object(126)
memory usage: 39.5+ KB


In [ ]:
df.columns

Index(['In The Box', 'Model Number', 'Model Name', 'Color', 'Browse Type',
       'SIM Type', 'Hybrid Sim Slot', 'Touchscreen', 'OTG Compatible',
       'Sound Enhancements',
       ...
       'DLNA Support', 'Mini HDMI Port', 'Dual Battery', 'Supported Languages',
       'Phone Book Memory', 'Mobile Tracker', 'Keypad Type', 'Games',
       'Music Player', 'Battery Type'],
      dtype='object', length=126)

In [ ]:
df['reviews'][0]

['Good phone for a normal usage. I gifted it to my mom works well for her.',
 'Product is nice also i ordered in 1st sale i like display and battery also camera is good its value for money smartphone Display 6.55 inch best part also 1st changed camera module looks primum ...',
 "The 6.5-inch screen is fairly decent, and is good enough for games and videos. You get Android 10 with Realme UI, which offers quite a lot of customisation options but has a lot of preloaded apps. The 13-megapixel rear camera takes decent shots, and there's also a depth sensor for portraits.",
 'Very good budget phone , its good affordable smartphone in this price range , best in class 5000mah battery with powerful gaming processor , overall fabulous package .',
 'I love realme and love you too Flipkart. Super 💯👌 mobile with super prize and specs',
 'Wow amazing product this price and value for money  and display is amazing nice mobile phone this price I am happy battery life is good',
 "This is amazing product